In [1]:
import json
with open('/content/sample_data/DataEngineeringQ2.json') as f:
    data = json.load(f)

Question 4

In [2]:
import pandas as pd
patient_details = [record['patientDetails'] for record in data]
df = pd.DataFrame(patient_details)
df = df.replace("", None)
missing_percentage = df.isnull().mean() * 100
missing_percentage = missing_percentage.round(2)
print(missing_percentage)

_id                   0.00
firstName             0.00
lastName             70.97
emailId             100.00
gender               32.26
alternateContact     96.77
birthDate            32.26
phrId                74.19
chat                 48.39
customId             48.39
dtype: float64


Question 5


In [3]:
gender_df = df[['gender']]
mode_gender = gender_df['gender'].mode()[0]
gender_df['gender'] = gender_df['gender'].fillna(mode_gender)
female_percentage = (gender_df['gender'] == 'F').mean() * 100
print(round(female_percentage, 2))

32.26


<ipython-input-3-404362d282ad>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_df['gender'] = gender_df['gender'].fillna(mode_gender)


Question 6

In [6]:
from datetime import datetime
current_year = datetime.now().year
def calculate_age(birth_date):
    if isinstance(birth_date, str): # Ensure it's a string
        birth_year = datetime.fromisoformat(birth_date.replace("Z", "")).year
        return current_year - birth_year
    return None

df['age'] = df['birthDate'].apply(calculate_age)
def categorize_age(age):
    if age is None:
        return None
    elif age <= 12:
        return 'Child'
    elif 13 <= age <= 19:
        return 'Teen'
    elif 20 <= age <= 59:
        return 'Adult'
    else:
        return 'Senior'
df['ageGroup'] = df['age'].apply(categorize_age)
adult_count = (df['ageGroup'] == 'Adult').sum()
print(f"Number of Adults: {adult_count}")

Number of Adults: 21


Question 7


In [7]:
medicines = [len(record['consultationData']['medicines']) for record in data]
average_medicines = sum(medicines) / len(medicines)
print(round(average_medicines, 2))

2.13


Question 8

In [8]:
from collections import Counter

all_medicines = [medicine['medicineName'] for record in data for medicine in record['consultationData']['medicines']]
medicine_counts = Counter(all_medicines)
third_most_common = medicine_counts.most_common(3)[2][0]
print(third_most_common)

C


Question 9

In [9]:
is_active = [medicine['isActive'] for record in data for medicine in record['consultationData']['medicines']]
total = len(is_active)
active_count = sum(is_active)
inactive_count = total - active_count
active_percentage = (active_count / total) * 100
inactive_percentage = (inactive_count / total) * 100
print(round(active_percentage, 2), round(inactive_percentage, 2))

69.7 30.3


Question 10

In [10]:
phone_numbers = [record['phoneNumber'] for record in data]
def is_valid_phone(phone):
    if phone.startswith('+91'):
        phone = phone[3:]
    elif phone.startswith('91'):
        phone = phone[2:]
    return phone.isdigit() and len(phone) == 10 and 6000000000 <= int(phone) <= 9999999999

valid_count = sum(is_valid_phone(phone) for phone in phone_numbers)
print(valid_count)

18


Question 11


In [14]:
from scipy.stats import pearsonr
ages = [calculate_age(record['patientDetails'].get('birthDate')) for record in data] # Using .get to handle missing keys
medicine_counts = [len(record['consultationData']['medicines']) for record in data]
valid_data = [(age, count) for age, count in zip(ages, medicine_counts) if age is not None]
age_values, medicine_values = zip(*valid_data)
correlation, _ = pearsonr(age_values, medicine_values)
print(round(correlation, 2))

-0.21
